<a href="https://colab.research.google.com/github/KhamessiTaha/ALPR/blob/main/CAR_MODEL_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
import os
import pandas as pd



In [2]:

# Mount Google Drive
drive.mount('/content/drive')




Mounted at /content/drive


In [7]:
!pip install rarfile


In [8]:
import os
import rarfile
import pandas as pd

# Define the directory containing the RAR files in your Google Drive
rar_directory = '/content/drive/MyDrive/dataset'

# Define the directory in your Google Drive where you want to extract the images
extracted_directory = '/content/drive/MyDrive/dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extracted_directory, exist_ok=True)

# Extract the contents of the RAR files
for filename in os.listdir(rar_directory):
    if filename.endswith('.rar'):
        # Get the full path to the RAR file
        rar_path = os.path.join(rar_directory, filename)

        # Create a folder to extract the contents into
        folder_name = os.path.splitext(filename)[0]
        extraction_folder = os.path.join(extracted_directory, folder_name)
        os.makedirs(extraction_folder, exist_ok=True)

        # Extract the contents of the RAR file into the extraction folder
        with rarfile.RarFile(rar_path, 'r') as rar_ref:
            rar_ref.extractall(extraction_folder)

print("RAR file extraction complete!")

# Load CSV files
train_labels = pd.read_csv(os.path.join(dataset_path, 'train_labels.csv'))
test_labels = pd.read_csv(os.path.join(dataset_path, 'test_labels.csv'))
val_labels = pd.read_csv(os.path.join(dataset_path, 'val_labels.csv'))

# Display the first few rows of the train_labels dataframe
print("Train Labels:")
print(train_labels.head())

# Display the first few rows of the test_labels dataframe
print("\nTest Labels:")
print(test_labels.head())

# Display the first few rows of the val_labels dataframe
print("\nValidation Labels:")
print(val_labels.head())



RAR file extraction complete!


NameError: name 'dataset_path' is not defined

In [9]:
# Load CSV files
train_labels = pd.read_csv(os.path.join(rar_directory, 'train_labels.csv'))
test_labels = pd.read_csv(os.path.join(rar_directory, 'test_labels.csv'))
val_labels = pd.read_csv(os.path.join(rar_directory, 'val_labels.csv'))

# Display the first few rows of the train_labels dataframe
print("Train Labels:")
print(train_labels.head())

# Display the first few rows of the test_labels dataframe
print("\nTest Labels:")
print(test_labels.head())

# Display the first few rows of the val_labels dataframe
print("\nValidation Labels:")
print(val_labels.head())

Train Labels:
        Image Make     Model  Year
0  657068.jpg  Kia  Sportage  2015
1  657050.jpg  Kia  Sportage  2015
2  656981.jpg  Kia  Sportage  2015
3  657192.jpg  Kia  Sportage  2015
4  657146.jpg  Kia  Sportage  2015

Test Labels:
        Image Make     Model  Year
0  657235.jpg  Kia  Sportage  2015
1  657010.jpg  Kia  Sportage  2015
2  657172.jpg  Kia  Sportage  2015
3  657229.jpg  Kia  Sportage  2015
4  657194.jpg  Kia  Sportage  2015

Validation Labels:
        Image Make     Model  Year
0  656975.jpg  Kia  Sportage  2015
1  657121.jpg  Kia  Sportage  2015
2  657110.jpg  Kia  Sportage  2015
3  657242.jpg  Kia  Sportage  2015
4  657009.jpg  Kia  Sportage  2015


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [11]:
# Define image dimensions
img_width, img_height = 300, 300

# Define batch size
batch_size = 32

In [12]:
# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [25]:
# ImageDataGenerator for validation and testing data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)


In [23]:

import os
import numpy as np
import pandas as pd

# Assuming 'train_labels' is your DataFrame containing the training labels
train_labels_array = train_labels[['Image', 'Make', 'Model', 'Year']].to_numpy()

# Specify the directory containing the training images
train_directory_array = os.path.join(rar_directory, 'train/train')

# Load and preprocess training data with NumPy arrays
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame(train_labels_array, columns=['Image', 'Make', 'Model', 'Year']),
    directory=train_directory_array,
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)


Found 1024742 validated image filenames.


In [26]:
# Convert lists to NumPy arrays if necessary
test_labels_array = test_labels[['Image', 'Make', 'Model', 'Year']].to_numpy()
val_labels_array = val_labels[['Image', 'Make', 'Model', 'Year']].to_numpy()

# Specify the directories containing the test and validation images
test_directory_array = os.path.join(rar_directory, 'test/test')
val_directory_array = os.path.join(rar_directory, 'val/val')

# Load and preprocess test data with NumPy arrays
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame(test_labels_array, columns=['Image', 'Make', 'Model', 'Year']),
    directory=test_directory_array,
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)

# Load and preprocess validation data with NumPy arrays
val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame(val_labels_array, columns=['Image', 'Make', 'Model', 'Year']),
    directory=val_directory_array,
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)


Found 224680 validated image filenames.
Found 217757 validated image filenames.


In [27]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [28]:
# Load pre-trained ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

94668760/94668760 [==============================] - 1s 0us/step


In [29]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

In [30]:
# Define new model with modified architecture
model = Model(inputs=base_model.input, outputs=predictions)

In [31]:
# Freeze base layers (optional)
for layer in base_model.layers:
    layer.trainable = False


In [32]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Display model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 306, 306, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 150, 150, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 152, 152, 64)         0         ['conv1_conv[0][0]']          
                                                                                              

In [33]:
epochs = 10


In [45]:
print(val_generator.samples)
print(train_generator.samples)
print(test_generator.samples)
print(batch_size)

217757
1024742
224680
32


In [37]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=int(train_generator.samples / batch_size),  # Convert to integer
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=int(val_generator.samples / batch_size),  # Convert to integer
    callbacks=[checkpoint, early_stopping]
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 300, 300, 3) <dtype: 'float32'>
(None, 3) <dtype: 'float32'>
input_1 [(None, 300, 300, 3)] float32
conv1_pad (None, 300, 300, 3) float32
conv1_conv (None, 306, 306, 3) float32
pool1_pad (None, 150, 150, 64) float32
pool1_pool (None, 152, 152, 64) float32
conv2_block1_preact_bn (None, 75, 75, 64) float32
conv2_block1_preact_relu (None, 75, 75, 64) float32
conv2_block1_1_conv (None, 75, 75, 64) float32
conv2_block1_1_bn (None, 75, 75, 64) float32
conv2_block1_1_relu (None, 75, 75, 64) float32
conv2_block1_2_pad (None, 75, 75, 64) float32
conv2_block1_2_conv (None, 77, 77, 64) float32
conv2_block1_2_bn (None, 75, 75, 64) float32
conv2_block1_2_relu (None, 75, 75, 64) float32
conv2_block1_0_conv (None, 75, 75, 64) float32
conv2_block1_3_conv (None, 75, 75, 64) float32
conv2_block1_out [(None, 75, 75, 256), (None, 75, 75, 256)] float32
conv2_block2_preact_bn (None, 75, 75, 256) float32
conv2_block2_preact_relu (None, 75, 75, 256) float32
conv2_block2_1_conv (None, 75, 75, 256) float3

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

#TRAIN THE MODEL


In [36]:
# Define number of epochs
epochs = 10

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# Evaluate the model on testing data
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
